<a href="https://colab.research.google.com/github/unshDee/voxart-backend/blob/main/voxart_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <center>Use this notebook to run your text-to-image server</center>
### <center> [VoxArt Backend Repository](https://github.com/unshDee/voxart-backend) </center>

## Install dependencies

In [ ]:
!git clone https://github.com/unshDee/voxart-backend.git
!pip install -r voxart-backend/backend/requirements.txt
!pip install diffusers["torch"]
!pip install -q transformers
!pip install -q xformers

# Run the backend web server

In [ ]:
from threading import Thread, Event

app_port = 8000
announce_url = None
cloudflared_startup = Event()

def update_announce_url(url):
    global announce_url
    announce_url = url

def start_cloudflared(port):
  from flask_cloudflared import _run_cloudflared
  try:
    announce_url = _run_cloudflared(port, 8888)
  except:
    raise
  finally:
    update_announce_url(announce_url)
    cloudflared_startup.set()

def run_with_cloudflared(thread):
    old_run = thread.run

    def new_run(*args, **kwargs):
        new_thread = Thread(target = start_cloudflared, args=(app_port, ))
        new_thread.setDaemon(True)
        new_thread.start()
        old_run(*args, **kwargs)

    thread.run = new_run

def app():
  !python voxart-backend/backend/app.py --port {app_port} --save_to_disk true --img_format jpeg --output_dir generations

if __name__ == '__main__':
    t1 = Thread(target = app)
    run_with_cloudflared(t1)
    t1.start()
    cloudflared_startup.wait()
    print(f"Your url is: {announce_url}")
    t1.join()

<ipython-input-7-5f61c1f667f6>:26: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  new_thread.setDaemon(True)


Your url is: https://editorials-hong-bangladesh-ata.trycloudflare.com
2023-05-29 00:26:23.587997: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/flax/struct.py:133: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/usr/local/lib/python3.10/dist-packages/flax/struct.py:133: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/usr/local/lib/python3.10/dist-packages/flax/struct.py:133: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
--> S

### Now, take the url you got from cloudflare (should look like `your url is: https://xxxxxx.trycloudflare.com`).

### Let the fun begin ✨